# 영어-한국어 번역기 만들기

## Load Dataset

In [1]:
from datasets import load_dataset

In [27]:
ko_en_dataset = load_dataset('Heerak/ko_en_translation', cache_dir='/data/heerak/.cache')

/home/heerak/anaconda3/envs/heerak/lib/python3.8/site-packages/huggingface_hub/utils/_hf_folder.py:92: UserWarning: A token has been found in `/home/heerak/.huggingface/token`. This is the old path where tokens were stored. The new location is `/home/heerak/.cache/huggingface/token` which is configurable using `HF_HOME` environment variable. Your token has been copied to this new location. You can now safely delete the old token file manually or use `huggingface-cli logout`.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7459339 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/740903 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /data/heerak/.cache/Heerak___parquet/Heerak--ko_en_translation-38d10c8228802b34/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
ko_en_dataset

DatasetDict({
    train: Dataset({
        features: ['ko', 'en', '__index_level_0__'],
        num_rows: 7459339
    })
    valid: Dataset({
        features: ['ko', 'en', '__index_level_0__'],
        num_rows: 740903
    })
})

In [29]:
print(ko_en_dataset['train']['ko'][:3])

['원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.', '형님 제일 웃긴 그림이 뭔지 알아요.', '속옷을?']


In [30]:
print(ko_en_dataset['train']['en'][:3])

['If you reply to the color you want, we will start making it right away.', 'You know what the funniest picture is.', 'Underwear?']


In [31]:
en_ko_dataset = load_dataset('Heerak/en_ko_translation', cache_dir='/data/heerak/.cache')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3326179 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/403802 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /data/heerak/.cache/Heerak___parquet/Heerak--en_ko_translation-5921884cd7d8cfd7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
en_ko_dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'ko', '__index_level_0__'],
        num_rows: 3326179
    })
    valid: Dataset({
        features: ['en', 'ko', '__index_level_0__'],
        num_rows: 403802
    })
})

In [33]:
print(en_ko_dataset['train']['ko'][:3])

['그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다.', '확실히 생각하고 있습니다만, 몇 가지 여쭤보고 싶은 게 있어요.', '오늘날 세계 5가구 중 1가구는 고양이나 개 또는 둘 다를 키우고 있습니다.']


In [34]:
print(en_ko_dataset['train']['en'][:3])

["I'm glad to hear that, and I hope you do consider doing business with us.", "I'm definitely thinking about it, but I have some queries to ask you.", "In today's world, one in every five families has either a cat, dog, or both."]


## Create Vocabulary

In [10]:
import os

In [11]:
en_ko_train_df = en_ko_dataset['train'].to_pandas()

In [12]:
en_ko_train_df.head()

,en,ko,__index_level_0__
0,"I'm glad to hear that, and I hope you do consi...","그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다.",0
1,"I'm definitely thinking about it, but I have s...","확실히 생각하고 있습니다만, 몇 가지 여쭤보고 싶은 게 있어요.",1
2,"In today's world, one in every five families h...",오늘날 세계 5가구 중 1가구는 고양이나 개 또는 둘 다를 키우고 있습니다.,2
3,"When you tell them, we'll take care of their c...","그들에게 말하면, 저희가 그 아이들을 돌볼 것입니다.",3
4,"OK, how about for swimming?","좋아요, 수영은 어떤가요?",4


In [13]:
ko_en_train_df = ko_en_dataset['train'].to_pandas()

In [14]:
ko_en_train_df.head()

,ko,en,__index_level_0__
0,원하시는 색상을 회신해 주시면 바로 제작 들어가겠습니다.,"If you reply to the color you want, we will st...",0
1,형님 제일 웃긴 그림이 뭔지 알아요.,You know what the funniest picture is.,1
2,속옷을?,Underwear?,2
3,그래도 가격이 꽤 비싸니까 많이 살게요.,I wont buy a lot though since the price is sti...,3
4,"AAA님, 제가 회의에서 화를 냈던 점 정말 사과드리고 싶습니다.","Dear AAA, I really want to apologize for my an...",4


### Make English Vocab

In [15]:
data_path = os.path.join(os.getcwd(), 'data')
data_path

'/data/heerak/workspace/translation_koenja/data'

In [16]:
if not os.path.exists(os.path.join(data_path, 'vocab')):
    os.makedirs(os.path.join(data_path, 'vocab'), exist_ok=True)
    
# vocab 을 만들기 위한 text file 만듦
with open(os.path.join(data_path, 'vocab', 'english.txt'), 'w', encoding='utf-8') as f:
    f.write('\n'.join(en_ko_train_df['en']))
    f.write('\n')
    f.write('\n'.join(ko_en_train_df['en']))

In [17]:
from tokenizers import BertWordPieceTokenizer

In [18]:
tokenizer = BertWordPieceTokenizer(
    strip_accents=False,  # True 일 경우 악센트 제거. ex) é → e, ô → o
    lowercase=False,  # 대소문자 구분 여부. True 일 경우 구분하지 않음
)

In [19]:
# tokenizer 학습시 필요한 parameter
data_file = os.path.join(data_path, 'vocab', 'english.txt')
vocab_size = 32000
limit_alphabet = 6000
min_frequency = 5

special_tokens = ['[PAD]', '[UNK]', '[SOS]', '[EOS]', '[MASK]']

unused_ids = 200
unused_special_token_list = ['[UNUSED{}]'.format(i) for i in range(unused_ids)]
special_tokens.extend(unused_special_token_list)

In [20]:
tokenizer.train(
    files=data_file,  # 단어 집합을 얻기 위해 학습할 데이터
    vocab_size=vocab_size,  # 단어 집합의 크기
    min_frequency=min_frequency,  # 최소 해당 횟수만큼 등장한 쌍(pair)의 경우에만 병합 대상이 됨
    limit_alphabet=limit_alphabet,  # 병합 전의 초기 토큰의 허용 개수
    special_tokens=special_tokens,  # special token 리스트
)

In [21]:
tokenizer

Tokenizer(vocabulary_size=32000, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=False, lowercase=False, wordpieces_prefix=##)

In [22]:
if not os.path.exists('./tokenizer/english'):
    os.makedirs('./tokenizer/english', exist_ok=True)
tokenizer.save_model('./tokenizer/english')

['./tokenizer/english/vocab.txt']

In [23]:
encoded = tokenizer.encode('I love you')
print('토큰화 결과 :', encoded.tokens)
print('정수 인코딩 :', encoded.ids)
print('디코딩 :', tokenizer.decode(encoded.ids))

토큰화 결과 : ['I', 'love', 'you']
정수 인코딩 : [244, 7869, 4153]
디코딩 : I love you


### Make Korean Vocab

In [24]:
if not os.path.exists(os.path.join(data_path, 'vocab')):
    os.makedirs(os.path.join(data_path, 'vocab'), exist_ok=True)
    
# vocab 을 만들기 위한 text file 만듦
with open(os.path.join(data_path, 'vocab', 'korean.txt'), 'w', encoding='utf-8') as f:
    f.write('\n'.join(en_ko_train_df['ko']))
    f.write('\n')
    f.write('\n'.join(ko_en_train_df['ko']))

In [25]:
tokenizer = BertWordPieceTokenizer(
    strip_accents=False,  # True 일 경우 악센트 제거. ex) é → e, ô → o
    lowercase=False,  # 대소문자 구분 여부. True 일 경우 구분하지 않음
)

In [26]:
# tokenizer 학습시 필요한 parameter
data_file = os.path.join(data_path, 'vocab', 'korean.txt')
vocab_size = 32000
limit_alphabet = 6000
min_frequency = 5

special_tokens = ['[PAD]', '[UNK]', '[SOS]', '[EOS]', '[MASK]']

unused_ids = 200
unused_special_token_list = ['[UNUSED{}]'.format(i) for i in range(unused_ids)]
special_tokens.extend(unused_special_token_list)

In [27]:
tokenizer.train(
    files=data_file,  # 단어 집합을 얻기 위해 학습할 데이터
    vocab_size=vocab_size,  # 단어 집합의 크기
    min_frequency=min_frequency,  # 최소 해당 횟수만큼 등장한 쌍(pair)의 경우에만 병합 대상이 됨
    limit_alphabet=limit_alphabet,  # 병합 전의 초기 토큰의 허용 개수
    special_tokens=special_tokens,  # special token 리스트
)

In [28]:
tokenizer

Tokenizer(vocabulary_size=32000, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=False, lowercase=False, wordpieces_prefix=##)

In [29]:
if not os.path.exists('./tokenizer/korean'):
    os.makedirs('./tokenizer/korean', exist_ok=True)
tokenizer.save_model('./tokenizer/korean')

['./tokenizer/korean/vocab.txt']

In [30]:
encoded = tokenizer.encode('오늘도 손흥민을 보며 하루를 시작하고 토트넘 우승을 향해 응원하자!')
print('토큰화 결과 :', encoded.tokens)
print('정수 인코딩 :', encoded.ids)
print('디코딩 :', tokenizer.decode(encoded.ids))

토큰화 결과 : ['오늘', '##도', '손흥', '##민을', '보며', '하루', '##를', '시작하고', '토트넘', '우승을', '향해', '응원', '##하자', '!']
정수 인코딩 : [9859, 6232, 17485, 19984, 28345, 10673, 6228, 28302, 25006, 23108, 12743, 19585, 11833, 205]
디코딩 : 오늘도 손흥민을 보며 하루를 시작하고 토트넘 우승을 향해 응원하자!


## Seed Setting

In [35]:
from transformers import set_seed

In [36]:
set_seed(42)

## Model Load

### Vanila Transformer

In [37]:
import math

import torch
import torch.nn as nn
import torch.optim as optim

from torch import Tensor

In [38]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 d_model: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(-torch.arange(0, d_model, 2) * math.log(10000) / d_model)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, d_model))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)
        
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)  # 일종의 layer 로서 작용하지만, optimizer 에 의해 업데이트 되지 않도록 함
        
    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])
    
    
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, d_model: int):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model
        
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.d_model)
    
    
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 d_model: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 2048,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = nn.Transformer(d_model=d_model, 
                                          nhead=nhead, 
                                          num_encoder_layers=num_encoder_layers, 
                                          num_decoder_layers=num_decoder_layers, 
                                          dim_feedforward=dim_feedforward, 
                                          dropout=dropout)
        self.linear = nn.Linear(d_model, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, d_model)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout)
        
    def forward(self,
                src: Tensor,
                tgt: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        
        # 토큰 임베딩을 거친 후 positional encoding 을 수행하여 임베딩 벡터를 만듦
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(tgt))
        
        # Seq2Seq 트랜스포머 네트워크를 통과함
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        
        # 디코더 결과를 linear layer 에 통과시켜 target vocab size 만큼 차원을 맞추어 줌
        return self.linear(outs)
    
    def encode(self, 
               src: Tensor,
               src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(self.src_tok_emb(src)), src_mask)
    
    def decode(self, 
               tgt: Tensor,
               memory: Tensor,
               tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask)

### Source(english) Tokenizer | Target(Korean) Tokenizer

In [39]:
from transformers import BertTokenizerFast

In [40]:
source_vocab_path = "./tokenizer/english/"
source_tokenizer = BertTokenizerFast.from_pretrained(source_vocab_path,
                                                     unk_token='[UNK]',
                                                     sep_token='[EOS]',
                                                     pad_token='[PAD]',
                                                     cls_token='[SOS]',
                                                     mask_token='[MASK]',
                                                     model_max_length=256)

target_vocab_path = "./tokenizer/korean/"
target_tokenizer = BertTokenizerFast.from_pretrained(target_vocab_path,
                                                     unk_token='[UNK]',
                                                     sep_token='[EOS]',
                                                     pad_token='[PAD]',
                                                     cls_token='[SOS]',
                                                     mask_token='[MASK]',
                                                     model_max_length=256)

## DataCollator

In [41]:
from transformers import PreTrainedTokenizerBase
from transformers.tokenization_utils_base import PaddingStrategy

In [42]:
class TransformerDataCollator:

    def __init__(self, is_test=False):
        self.is_test = is_test

    def __call__(self, batch):
        b_src = []
        if not self.is_test:
            b_tgt = []
            b_labels = []
        
        for b in batch:
            b_src.append(b['src_input_ids'])
            
            if not self.is_test:
                b_tgt.append(b['tgt_input_ids'])
                b_labels.append(b['labels'])
                
        t_src = torch.LongTensor(b_src)  # List[Tensor] -> Tensor List
        if self.is_test:
            return {'src': t_src}
        else:
            t_tgt = torch.LongTensor(b_tgt)
            t_labels = torch.LongTensor(b_labels)
            return {'src': t_src, 'tgt': t_tgt, 'labels': b_labels}
        
        
        return features

## DataLoader

In [43]:
from torch.utils.data import DataLoader

In [44]:
def get_dataloader(dataset, src_lang, tgt_lang, src_tokenizer, tgt_tokenizer):
    train_dataset = dataset['train']
    valid_dataset = dataset['valid']
    
    data_collator = TransformerDataCollator()
    
    def preprocess_function(examples):
        inputs = [ex for ex in examples[src_lang]]
        targets = [ex for ex in examples[tgt_lang]]
        tokenized_inputs = src_tokenizer(inputs, max_length=256, padding='max_length', truncation=True)

        # Tokenize targets with the `text_target` keyword argument
        tokenized_targets = tgt_tokenizer(text_target=targets, max_length=256, padding='max_length', truncation=True)

        result = {}
        result['src_input_ids'] = tokenized_inputs['input_ids']
        result['tgt_input_ids'] = tokenized_targets['input_ids']
        
        # if we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        result['labels'] = [[(t if t != tgt_tokenizer.pad_token_id else -100) for t in tokenized_target] 
                            for tokenized_target in tokenized_targets['input_ids']]
        
        return result
    
    train_dataset = train_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=train_dataset.column_names,
    )
    valid_dataset = valid_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=valid_dataset.column_names,
    )
    
    train_dataloader = DataLoader(train_dataset,
                                  collate_fn=data_collator,
                                  shuffle=True,
                                  batch_size=batch_size)
    
    valid_dataloader = DataLoader(valid_dataset,
                                  collate_fn=data_collator,
                                  shuffle=False,
                                  batch_size=batch_size)
    
    return {'train': train_dataloader,
            'valid': valid_dataloader}

## Trainer

In [45]:
class Trainer:
    
    def __init__(self, model, optimizer, loss_fn, batch_size, src_lang, tgt_lang, device, src_tokenizer, tgt_tokenizer, dataset):
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.batch_size = batch_size
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang
        self.device = device
        
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer
        
        dataloader = get_dataloader(dataset, src_lang, tgt_lang, src_tokenizer, tgt_tokenizer)
        
        self.train_dataloader = dataloader['train']
        self.valid_dataloader = dataloader['valid']
        
    def train_epoch(self):
        self.model.train()
        losses = 0
        
        for batch in tqdm(self.train_dataloader):
            src = batch['src'].to(self.device)
            tgt = batch['tgt'].to(self.device)
            
            tgt_input = tgt[:-1, :]  # batch 에 있는 단어에서 마지막 [EOS] 토큰 제거
            
            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = self.create_mask(src, tgt_input)
            
            logits = self.model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
            
            self.optimizer.zero_grad()
            
            tgt_out = tgt[1:, :]  # batch 안에 있는 단어에서 [SOS] 토큰을 제거한 정답 sequence
            
            # Transformer Network 의 teacher forcing 을 아래와 같이 구현함
            # 즉, 학습시 auto regressive 한 디코더의 특징을 활용하는 것이 아니라,
            # t-1 시점의 예측과 무관히 t 시점 디코더에는 ground truth 토큰을 입력함
            
            # tgt_input 은 [SOS] 토큰으로 '정답 sequence 인 tgt_out 의 첫 토큰'을 맞추려 함
            # tgt_input 의 마지막 토큰으로 '정답 sequence 인 tgt_out 의 마지막 토큰'인 [EOS] 를 맞추려 함
            loss = self.loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            loss.backward()
            
            self.optimizer.step()
            losses += loss.item()
            
        return losses / len(self.train_dataloader)
    
    def evaluate(self):
        self.model.eval()
        losses = 0
        
        for batch in tqdm(self.valid_dataloader):
            with torch.no_grad():
                src = batch['src'].to(self.device)
                tgt = batch['tgt'].to(self.device)
                labels = batch['labels'].to(self.device)

                tgt_input = tgt[:-1, :]  # batch 에 있는 단어에서 마지막 [EOS] 토큰 제거

                src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = self.create_mask(src, tgt_input)

                logits = self.model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

                tgt_out = labels[1:, :]  # batch 안에 있는 단어에서 [SOS] 토큰을 제거한 정답 sequence

                # tgt_input 은 [SOS] 토큰으로 '정답 sequence 인 tgt_out 의 첫 토큰'을 맞추려 함
                # tgt_input 의 마지막 토큰으로 '정답 sequence 인 tgt_out 의 마지막 토큰'인 [EOS] 를 맞추려 함
                loss = self.loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
                losses += loss.item()
            
            # valid 에 대해서는 backward propagation 을 수행하지 않음
        
        return losses / len(self.valid_dataloader)
            
            
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones((sz, sz), device=self.device)) == 1).transpose(0, 1)
        
        """
        (eg.) 타겟 시퀀스 길이가 5 토큰이라고 가정하면,
        >>> mask
        tensor([[True, False, False, False, False],  # 첫 토큰은 자기 자신만 attend 할 수 있음
                [True,  True, False, False, False],
                [True,  True,  True, False, False],
                [True,  True,  True,  True, False],
                [True,  True,  True,  True,  True]])  # 마지막 토큰은 모든 토큰을 attend 할 수 있음
        """
        
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        """
        (eg.) 타겟 시퀀스 길이가 5 토큰이라고 가정하면,
        >>> mask
        tensor([[0, -inf, -inf, -inf, -inf],  # 첫 토큰은 자기 자신만 attend 할 수 있음
                [0,    0, -inf, -inf, -inf],
                [0,    0,    0, -inf, -inf],
                [0,    0,    0,    0, -inf],
                [0,    0,    0,    0,    0]])  # 마지막 토큰은 모든 토큰을 attend 할 수 있음
        """
        
        return mask
        
            
    def create_mask(self, src, tgt):
        """ mask 되지 않고 attend 할 수 있는 토큰을 False 또는 0 값으로 이루어지도록 처리함 """
        src_seq_len = src.shape[0]
        tgt_seq_len = tgt.shape[0]
        
        tgt_mask = self.generate_square_subsequent_mask(tgt_seq_len)
        src_mask = torch.zeros((src_seq_len, src_seq_len), device=self.device).type(torch.bool)
        # src_mask 는 (src 시퀀스길이 X src 시퀀스길이)의 False 들로 이루어진 행렬임)
        
        src_padding_mask = (src == self.src_tokenizer.pad_token_id).transpose(0, 1)
        tgt_padding_mask = (tgt == self.tgt_tokenizer.pad_token_id).transpose(0, 1)
        
        """
        (eg.)
        src_padding_mask 및 tgt_padding_mask 는 아래와 같이 batch 로 묶인 데이터들에 대해서 pad_token_id 에 해당하는 부분은 False 로 처리함
        
        tensor([[False, False, False, False, False, True, True, True, True],
                [False, False, False, False, True, True, True, True, True],
                [False, False, False, False, False, False, False, False, True]])
        """
        
        return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

## Training

In [46]:
# Model Train Parameter
num_epochs = 30
num_encoder_layers = 6
num_decoder_layers = 6
d_model = 512
nhead = 8
dim_feedforward = 2048
batch_size = 16
src_vocab_size = source_tokenizer.vocab_size
tgt_vocab_size = target_tokenizer.vocab_size
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [47]:
source_tokenizer.vocab_size

32000

In [48]:
transformer = Seq2SeqTransformer(num_encoder_layers=num_encoder_layers, 
                                 num_decoder_layers=num_decoder_layers, 
                                 d_model=d_model, 
                                 nhead=nhead, 
                                 src_vocab_size=src_vocab_size, 
                                 tgt_vocab_size=tgt_vocab_size,
                                 dim_feedforward=dim_feedforward)
transformer.to(device)

Seq2SeqTransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_featu

In [49]:
# xavier_uniform 파라미터 초기화
for n, p in transformer.named_parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [50]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)

In [51]:
trainer = Trainer(model=transformer,
                  optimizer=optimizer,
                  loss_fn=loss_fn, 
                  batch_size=batch_size,
                  src_lang='en',
                  tgt_lang='ko',
                  device=device,
                  src_tokenizer=source_tokenizer,
                  tgt_tokenizer=target_tokenizer,
                  dataset=en_ko_dataset,
                  )

Map:   0%|          | 0/3326179 [00:00<?, ? examples/s]

Map:   0%|          | 0/403802 [00:00<?, ? examples/s]

In [52]:
import os
import numpy as np

from timeit import default_timer as timer
from tqdm import tqdm

In [53]:
log_dir = './log/transformer'
if not os.path.exists(log_dir):
    os.makedirs(log_dir, exist_ok=True)
    
save_dir = './saved_model/transformer'
if not os.path.exists(save_dir):
    os.makedirs(save_dir, exist_ok=True)

In [54]:
train_losses = []
valid_losses = []

for epoch in range(1, num_epochs+1):
    start_time = timer()
    train_loss = trainer.train_epoch()
    end_time = timer()
    
    valid_loss = trainer.evaluate()
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    print(f'Epoch: {epoch}, Train loss: {train_loss:.3f}, Valid loss: {valid_loss:.3f}, Epoch time = {(end_time - start_time):.3f}s')
    
    # 에폭마다 loss 의 History 를 남김
    np.savetxt(os.path.join(log_dir, 'loss_history.txt'), np.array([train_losses, valid_losses]), fmt='%.4e')
    
    if epoch == 1:
        LEAST_VALID_LOSS = valid_loss
        torch.save({'model': transformer.state_dict(),
                    'optimizer': optimizer.state_dict()},
                   os.path.join(save_dir, 'best_model.pt'))
        source_tokenizer.save_pretrained(save_dir)
        target_tokenizer.save_pretrained(save_dir)
    else:
        if LEAST_VALID_LOSS > valid_loss:
            print('모델 갱신 : valid loss {}'.format(valid_loss))
            LEAST_VALID_LOSS = valid_loss
        torch.save({'model': transformer.state_dict(),
                    'optimizer': optimizer.state_dict()},
                   os.path.join(save_dir, 'best_model.pt'))
        source_tokenizer.save_pretrained(save_dir)
        target_tokenizer.save_pretrained(save_dir)

 42%|██████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                                  | 87319/207887 [6:30:43<7:22:55,  4.54it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 49%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                 | 102467/207887 [7:26:09<6:25:05,  4.56it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To chang

In [57]:
train_losses

[]